In [11]:
# from qutip import *
from math import sqrt
import numpy as np
import heapq as hq
from scipy.linalg import sqrtm
from scipy import sparse
import scipy.linalg as la
from scipy import *
import time
import matplotlib.pyplot as plt
import copy
import os.path
import qgeo as q

import matplotlib.pyplot as plt
import scipy.stats as stat
import matplotlib as mpl
import numpy as np
import matplotlib.mlab as mlab
import math
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D


p=3

#Generate arbitrary matrix
def matrix(n):
    A = np.arange(n**4).reshape(n**2,n**2)
    return A

#def matrix_multiplication(A,B)

# generate random unitary of size nxn close to the identity
def random_mixed_state_bipartite(n):
    
	# generate random complex matrix A
	A = np.random.randn(n**2,n**2)+complex(0,1)*np.random.randn(n**2,n**2)
	
	# compute A*A^dagger which is positive semidefinite
	A=np.matmul(A.conj().T,A)
	
	# normalize by trace
	A/=np.trace(A)
	
	return A

def random_mixed_state(n):
    
	# generate random complex matrix A
	A = np.random.randn(n,n)+complex(0,1)*np.random.randn(n,n)
	
	# compute A*A^dagger which is positive semidefinite
	A=np.matmul(A.conj().T,A)
	
	# normalize by trace
	A/=np.trace(A)
	
	return A

def partial_transpose_1xT(A,n):
    A=np.moveaxis(A.reshape(n,n,n,n),[1,3],[3,1]).reshape(n**2,n**2)
    return A

def partial_transpose_Tx1(A,n):
    A=np.moveaxis(A.reshape(n,n,n,n),[0,2],[2,0]).reshape(n**2,n**2)
    return A

def negativity(A):
    evals,evects=np.linalg.eig(A)
    negativity=np.trace(np.diag(abs(evals))-np.diag(evals))
    return negativity

def PPT_check(A,n):
    a=0
    A=partial_transpose_1xT(A,n)
    neg=np.round(negativity(A),10)
    if neg != 0: #If the negativity of the partial transpose is 0, i.e., if the state is PPT (not detected by PPT criterion), then a is 1
        a+=1
    return a

def identity(n):
    T=np.ones(n)
    S=np.diag(T)
    return S

def felix_map_2(R,A,n):#R with size n**2 x n**2, A with size n x n
    IdxA=q.tensor_mix(identity(n), [0], A, [1], d=n) #9x9
    P=q.ptrace(np.matmul(R,IdxA), [1], d=n)
    G=np.matmul(IdxA,q.tensor_mix(P, [0], identity(n), [1], d=n))
    
    #P=np.trace(np.matmul(R,IdxA))
    #G=P*IdxA
    
    H=np.matmul(IdxA,np.matmul(R,IdxA)) #First subsystem is labeled as 0 and second subsystem as 1
    F=G-H
    return F

def A_map_1(R,A,na,nb):#R with size n**2 x n**2, A with size n x n
    AxId=q.tensor_mix(A, [0], identity(nb), [1], d=[na,nb])
    P=q.ptrace(np.matmul(AxId,R), [0], d=[na,nb])
    G=q.np.kron(A,P)
    #P=np.trace(np.matmul(R,IdxA))
    #G=P*IdxA
    H=np.matmul(AxId,np.matmul(R,AxId)) #Probably I should put A.conj().T if A is only positive in absolute value, |<psi|A|psi>|>=0
    F=G-H
    return F

def A_map_2(R,A,na,nb):#R with size n**2 x n**2, A with size n x n
    IdxA=q.tensor_mix(identity(na), [0], A, [1], d=[na,nb])
    P=q.ptrace(np.matmul(R,IdxA), [1], d=[na,nb])
    G=q.np.kron(P,A)
    #P=np.trace(np.matmul(R,IdxA))
    #G=P*IdxA
    H=np.matmul(IdxA,np.matmul(R,IdxA)) #Probably I should put A.conj().T if A is only positive in absolute value, |<psi|A|psi>|>=0
    F=G-H
    return F

def A_map_check(R,A,n):
    a=0
    neg1=np.round(negativity(A_map_1(R,A,n,n)),10)
    neg2=np.round(negativity(A_map_2(R,A,n,n)),10)
    if neg1 != 0 or neg2 != 0: #If the negativity of the state after applying F is not 0, i.e., if the entanglement of the state is detected by Felix criterion, then a is 1
        a+=1
    return a    

def partial_det(A,n):#Matrix compression map: partial determinant of subsystem 2, useful to see any matrix as a block matrix, eg to compute partial trace, this kinda shit, and then comp

    detB=np.zeros((n,n),dtype = 'complex_')
    B=np.zeros((n,n,n,n),dtype = 'complex_')
    #print(B)
    #A=np.arange(p**4)#0,1,2,...,8//9,10,11,...,etc
    #A=identity(n**2)
    A=A.reshape((n**2,n**2))
    #print(A)
    #print("A_01= "+str(A[0][1]))
    for k in range(n):
        for l in range(n):
            for i in range(n):
                for j in range(n):
                    B[k][l][i][j]=A[i+k*n][j+l*n]
                    detB[k][l]=np.linalg.det(B[k][l])
                
    #print(B)
    #print(B[1][2])
    #print(np.linalg.det(B[0][0]))
    #print(detB)
    Bcomp=identity(n)/factorial(n)-detB#np.ones((n,n))/(n)#Calculate matrix compression, 1-partialdeterminant, which is positive for A separable
    return Bcomp    
    
def partial_det_check(A,n):
    a=0
    neg=np.round(negativity(partial_det(A,n)),10)
    if neg != 0: #If the negativity of the state after applying F is not 0, i.e., if the entanglement of the state is detected by Felix criterion, then a is 1
        a+=1
    return a    
def factorial(n):
    a=n
    for i in range (1,n):
        a=a*(n-i)        
    return a

#Entangled states detected by PPT
EDppt=0
#Entangled states detected by Reshuffling
EDres=0
#Entangled states detected by Psi_A
EDpsiA=0

#Counter of states which are PPT (not detected by PPT criterion)
Cppt=0
#Counter of states which are PPT but detected by Felix
Cppt_but_felix=0
#Set the matrix A of Felix map
#A=identity(p)
#A=np.ones((p,p))#ket(+)bra(+)
#A=np.diag((2.,3.,1.))#ket(0)bra(0)




#A=random_mixed_state(2)
#B=np.array([[0]])#[[0,0,0],[0,0,0],[0,0,0]]
#A=q.direct_sum(A,B)
#A=A.reshape(p,p)
#A=np.matmul(A,A)#.conj().T
#print(A)
#evals,evects=np.linalg.eig(A)
#print(evals)

#A=np.diag((1.,0.,0.,0.,0.,0.,0.,0.))
#print(A)
#A=np.diag((3.,2.,1.))

#A/=np.trace(A)



file = open("DataParam40dx1i.txt","w") 


iterations=400
disc=400
t=[]
x=[]
y=[]
z=[]
for a in range (disc+1):
    for b in range (disc+1):     
        for c in range (disc+1):
            if a+b+c==disc: #if l!=0 and s!=0 and r!=0:
                ll=(a+0.)/disc
                ss=(b+0.)/disc
                rr=(c+0.)/disc
                
                u=(ll-ss)/2+0.5
                v=rr#-1.+1.
                
                A=np.diag((ll,ss,rr))
                A/=np.trace(A)
                #print(A)
                #x.append(l)
                #y.append(s)
                #z.append(r)
                for i in range (iterations):
                #Check that things work with arbitrary matrix
                #    print("rho: " + str(i))
                #    print(matrix(d))
                #    print(partial_transpose_1xT(matrix(d),d))
                #    print(partial_transpose_Tx1(matrix(d),d))
                #    print("negativity: "+str(negativity(matrix(d))))
                #    print(PPT_check(matrix(d),d))
                #Program itself
                    
                    #rho=q.tensor_mix(random_mixed_state(p), [0], random_mixed_state(p), [1], d=p)
                    rho=random_mixed_state_bipartite(p)
                    #rho=np.ones((p**2,p**2))
                    #rho=[[1.,0,0,1.],[0,0,0,0],[0,0,0,0],[1.,0,0,1.]] #Just checked for a Bell state and A equals Id that the program does what I want it to do
                    #print(rho)
                    #print(np.trace(rho))
                    rho/=np.trace(rho)
                    #print(rho)
                    #print(felix_map_2(rho,A,p))
                    #evals,evects=np.linalg.eig(rho) #Just diagonalize to print it afterwards
                    #print(np.round(evals,10))
                    #print(np.round(negativity(rho),10))
                    if PPT_check(rho,p)!=0:#if negativity of the partial transpose is 0
                        it=i*100/iterations
                        #print("PPT %"+str(it))
                        Cppt+=1
                        As=1
                        for j in range (As):
                            #A=random_mixed_state(p)
                            #print(A)
                            #print(rho)
                            #print(felix_map_2(rho,A,p))
                            if A_map_check(rho,A,p)!=0:#partial_det_check(rho,p)
                                #print("PPT but detected by our map! ")
                                Cppt_but_felix+=1
                                
                #x.append(ll)
                #y.append(ss)
                #z.append(rr)
                #t.append(100*Cppt_but_felix/iterations)
                
                file.write("%s \t\t %s \t\t %s \n" % (u,v,100*Cppt_but_felix/iterations))
    
                #print("Number of PPT states: "+str(Cppt))
                #plt.figure(1)
                #plt.plot(x[ll], y[ss], c=t[Cppt_but_felix])#, marker='o', color=cmap(j/float(particules))
                
                #print("Number of PPT states detected by our map: "+str(Cppt_but_felix))
                Cppt=0 #just setting Cnpt to 0 again, otherwise it keeps summing after each run
                Cppt_but_felix=0
                
file.close()           
               
#plt.show()







#Brute force: checking all A diagonal
"""

A=np.diag((1.,1.,0.,0.,0.,0.,0.,0.))
print(A)
#A=np.diag((3.,2.,1.))

#A/=np.trace(A)
iterations=100000
for i in range (iterations):
#Check that things work with arbitrary matrix
#    print("rho: " + str(i))
#    print(matrix(d))
#    print(partial_transpose_1xT(matrix(d),d))
#    print(partial_transpose_Tx1(matrix(d),d))
#    print("negativity: "+str(negativity(matrix(d))))
#    print(PPT_check(matrix(d),d))
#Program itself
    
    #rho=q.tensor_mix(random_mixed_state(p), [0], random_mixed_state(p), [1], d=p)
    rho=random_mixed_state_bipartite(p)
    #rho=np.ones((p**2,p**2))
    #rho=[[1.,0,0,1.],[0,0,0,0],[0,0,0,0],[1.,0,0,1.]] #Just checked for a Bell state and A equals Id that the program does what I want it to do
    #print(rho)
    #print(np.trace(rho))
    rho/=np.trace(rho)
    #print(rho)
    #print(felix_map_2(rho,A,p))
    #evals,evects=np.linalg.eig(rho) #Just diagonalize to print it afterwards
    #print(np.round(evals,10))
    #print(np.round(negativity(rho),10))
    if PPT_check(rho,p)!=0:#if negativity of the partial transpose is 0
        it=i*100/iterations
        #print("PPT %"+str(it))
        Cppt+=1
        As=1
        for j in range (As):
            #A=random_mixed_state(p)
            #print(A)
            #print(rho)
            #print(felix_map_2(rho,A,p))
            if felix_map_2_check(rho,A,p)!=0:#partial_det_check(rho,p)
                #print("PPT but detected by our map! ")
                Cppt_but_felix+=1

print("Number of PPT states: "+str(Cppt))
print("Number of PPT states detected by our map: "+str(Cppt_but_felix))
Cppt=0 #just setting Cnpt to 0 again, otherwise it keeps summing after each run
Cppt_but_felix=0

A=np.diag((1.,1.,1.,0.,0.,0.,0.,0.))
print(A)
#A=np.diag((3.,2.,1.))

#A/=np.trace(A)
iterations=100000
for i in range (iterations):
#Check that things work with arbitrary matrix
#    print("rho: " + str(i))
#    print(matrix(d))
#    print(partial_transpose_1xT(matrix(d),d))
#    print(partial_transpose_Tx1(matrix(d),d))
#    print("negativity: "+str(negativity(matrix(d))))
#    print(PPT_check(matrix(d),d))
#Program itself
    
    #rho=q.tensor_mix(random_mixed_state(p), [0], random_mixed_state(p), [1], d=p)
    rho=random_mixed_state_bipartite(p)
    #rho=np.ones((p**2,p**2))
    #rho=[[1.,0,0,1.],[0,0,0,0],[0,0,0,0],[1.,0,0,1.]] #Just checked for a Bell state and A equals Id that the program does what I want it to do
    #print(rho)
    #print(np.trace(rho))
    rho/=np.trace(rho)
    #print(rho)
    #print(felix_map_2(rho,A,p))
    #evals,evects=np.linalg.eig(rho) #Just diagonalize to print it afterwards
    #print(np.round(evals,10))
    #print(np.round(negativity(rho),10))
    if PPT_check(rho,p)!=0:#if negativity of the partial transpose is 0
        it=i*100/iterations
        #print("PPT %"+str(it))
        Cppt+=1
        As=1
        for j in range (As):
            #A=random_mixed_state(p)
            #print(A)
            #print(rho)
            #print(felix_map_2(rho,A,p))
            if felix_map_2_check(rho,A,p)!=0:#partial_det_check(rho,p)
                #print("PPT but detected by our map! ")
                Cppt_but_felix+=1

print("Number of PPT states: "+str(Cppt))
print("Number of PPT states detected by our map: "+str(Cppt_but_felix))
Cppt=0 #just setting Cnpt to 0 again, otherwise it keeps summing after each run
Cppt_but_felix=0

A=np.diag((1.,1.,1.,1.,1.,1.))
print(A)
#A=np.diag((3.,2.,1.))

#A/=np.trace(A)
iterations=100000
for i in range (iterations):
#Check that things work with arbitrary matrix
#    print("rho: " + str(i))
#    print(matrix(d))
#    print(partial_transpose_1xT(matrix(d),d))
#    print(partial_transpose_Tx1(matrix(d),d))
#    print("negativity: "+str(negativity(matrix(d))))
#    print(PPT_check(matrix(d),d))
#Program itself
    
    #rho=q.tensor_mix(random_mixed_state(p), [0], random_mixed_state(p), [1], d=p)
    rho=random_mixed_state_bipartite(p)
    #rho=np.ones((p**2,p**2))
    #rho=[[1.,0,0,1.],[0,0,0,0],[0,0,0,0],[1.,0,0,1.]] #Just checked for a Bell state and A equals Id that the program does what I want it to do
    #print(rho)
    #print(np.trace(rho))
    rho/=np.trace(rho)
    #print(rho)
    #print(felix_map_2(rho,A,p))
    #evals,evects=np.linalg.eig(rho) #Just diagonalize to print it afterwards
    #print(np.round(evals,10))
    #print(np.round(negativity(rho),10))
    if PPT_check(rho,p)!=0:#if negativity of the partial transpose is 0
        it=i*100/iterations
        #print("PPT %"+str(it))
        Cppt+=1
        As=1
        for j in range (As):
            #A=random_mixed_state(p)
            #print(A)
            #print(rho)
            #print(felix_map_2(rho,A,p))
            if felix_map_2_check(rho,A,p)!=0:#partial_det_check(rho,p)
                #print("PPT but detected by our map! ")
                Cppt_but_felix+=1

print("Number of PPT states: "+str(Cppt))
print("Number of PPT states detected by our map: "+str(Cppt_but_felix))
Cppt=0 #just setting Cnpt to 0 again, otherwise it keeps summing after each run
Cppt_but_felix=0


"""
    


    
    


KeyboardInterrupt: 